In [82]:
# NOTES FOR COMPILATION IN THE CLUSTER
# Here's the list of packages that we need to import:
# numpy, glob2, matplot, pymatreader, mne

# Notes that are specified for the cluster will have comments next to the word "CLUSTER".

# glob2: accessing the files in the folder
# pymatreader: access the visual graph of EEG (optional for cluster compilation)

from glob import glob
import os
import mne
import numpy as np
import matplotlib.pyplot as plt
import random

In [83]:
# Array that stores all the set files for music thinking

# CLUSTER: The path needs to be changed ('def-sponsor00/*.set' for example)
#          The symbol * means any sequence of characters.

all_eeg_path = glob('Music Thinking/sub-*/ses-*/eeg/*.set')
len(all_eeg_path)

240

In [84]:
# Array for the labels

# CLUSTER: The path needs to be changed as well
behaviour_data = np.genfromtxt('Music Thinking/stimuli/Behavioural_data.txt')
behaviour_data = behaviour_data[1:]

labels = behaviour_data[:,2]
for i in range(labels.shape[0]):
    # Enjoyment level 1 or 2 -> enjoy the most
    if labels[i] <= 2:
        labels[i] = 1
        
    # Neutral or does not enjoy (enjoyment levels 3 to 5)
    else:
        labels[i] = 0
        
# The proportion of people liking/disliking the music is relatively the same.
print(np.count_nonzero(labels == 1)) # LIKE
print(np.count_nonzero(labels == 0)) # DISLIKE
#labels

112
128


In [85]:
# Associate each path to the corresponding class (binary classification)
like_path = []
dislike_path = []

for i in range(labels.shape[0]):
    if labels[i] == 1:
        like_path = np.append(like_path, all_eeg_path[i])
    else:
        dislike_path = np.append(dislike_path, all_eeg_path[i])

# Test: 5 paths
like_path = like_path[0:5]
dislike_path = dislike_path[0:5]
        
print(like_path.shape[0], dislike_path.shape[0])

5 5


In [86]:
# Function that reads EEG Data (set extension file)
def read_set_data(path):
    music_data = mne.io.read_raw_eeglab(path, preload = True)
    #music_data.set_eeg_reference()
    
    epochs = mne.make_fixed_length_epochs(music_data, duration = 3, overlap = 2, preload = True)
    music_array = epochs.get_data()
    print(f'Dimensions of music_array: {music_array.shape}')
    
    number_epochs = 4
    array_epochs = np.empty(number_epochs, dtype = object)

    for i in range(number_epochs):
        chosen_number = random.randint(0, array.shape[0]-1)
        print(chosen_number)
        array_epochs[i] = array[chosen_number]
    
    return array_epochs

In [87]:
%%capture
random.seed(0)

like_epoch_array = np.empty((len(like_path)), dtype = object)
dislike_epoch_array = np.empty((len(dislike_path)), dtype = object)

for i in range(len(like_path)):
    like_epoch_array[i] = read_set_data(like_path[i])
    
for i in range(len(dislike_path)):
    dislike_epoch_array[i] = read_set_data(dislike_path[i])

In [88]:
like_epoch_array[0].shape

(4,)

In [89]:
# Assign the labels for each epoch
like_epoch_labels = np.empty((len(like_path)), dtype = object)
dislike_epoch_labels = np.empty((len(dislike_path)), dtype = object)

for i in range(len(like_path)):
    like_epoch_labels[i] = len(like_epoch_array[i]) * [1]
    
for i in range(len(dislike_path)):
    dislike_epoch_labels[i] = len(dislike_epoch_array[i]) * [0]
    
print(len(like_epoch_labels), len(dislike_epoch_labels))

5 5


In [90]:
# Assign the individuals for each epoch
data_list = np.append(like_epoch_array, dislike_epoch_array)
label_list = np.append(like_epoch_labels, dislike_epoch_labels)

# 10 individuals, 4 epochs per individual, (129 x 750) 

In [91]:
person_list = np.empty((10), dtype = object)
for i in range(len(person_list)):
    person_list[i] = len(data_list[i]) * [i]
    
print(len(person_list))

10


In [79]:
#label_list = np.hstack(label_list)
#person_list = np.hstack(person_list)
#data_list = np.hstack(data_list)

In [116]:
X_train = np.append(like_epoch_array[0:3], dislike_epoch_array[0:3])
X_val = np.append(like_epoch_array[3:4], dislike_epoch_array[3:4])
X_test = np.append(like_epoch_array[4:5], dislike_epoch_array[4:5])

y_train = np.append(like_epoch_labels[0:3], dislike_epoch_labels[0:3])
y_val = np.append(like_epoch_labels[3:4], dislike_epoch_labels[3:4])
y_test = np.append(like_epoch_labels[4:5], dislike_epoch_labels[4:5])